In [1]:
import pandas as pd              
import numpy as np 
import tensorflow as tf

data = pd.read_csv("https://raw.githubusercontent.com/spyroy/deep-learning-bible-classification/main/bible_data_set%20(with%20count%20and%20testament).csv")
data

,citation,book,chapter,verse,text,length (charcters),words,testament
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,56,10,old
1,Genesis 1:2,Genesis,1,2,"And the earth was without form, and void; and ...",144,29,old
2,Genesis 1:3,Genesis,1,3,"And God said, Let there be light: and there wa...",56,11,old
3,Genesis 1:4,Genesis,1,4,"And God saw the light, that it was good: and G...",87,17,old
4,Genesis 1:5,Genesis,1,5,"And God called the light Day, and the darkness...",117,22,old
...,...,...,...,...,...,...,...,...
31097,Revelation 22:17,Revelation,22,17,"And the Spirit and the bride say, Come. And le...",170,33,new
31098,Revelation 22:18,Revelation,22,18,For I testify unto every man that heareth the ...,189,37,new
31099,Revelation 22:19,Revelation,22,19,And if any man shall take away from the words ...,209,44,new
31100,Revelation 22:20,Revelation,22,20,"He which testifieth these things saith, Surely...",97,16,new


In [2]:
import nltk
from nltk.stem import PorterStemmer

ps = PorterStemmer() 

vocabulary_size = 0
word2location = {}

def prepare_vocabulary(data):
    index = 0
    for sentance in data['text']:
        #sentance = sentance.lower()
        words = nltk.word_tokenize(sentance)
        for word in words:
            stemed_word = ps.stem(word)
            if stemed_word not in word2location:
                word2location[stemed_word] = index
                index += 1
    return index

def convert2vec(sentance):
    #sentance = sentance.lower()
    res_vec = np.zeros(vocabulary_size)
    words = nltk.word_tokenize(sentance)
    for word in words:
        stemed_word = ps.stem(word)
        if stemed_word in word2location:
            res_vec[word2location[stemed_word]]+=1
    return res_vec

vocabulary_size = prepare_vocabulary(data)
print("the size of the vocabulary is: ", vocabulary_size)
word2location

the size of the vocabulary is:  9421


{'In': 0,
 'the': 1,
 'begin': 2,
 'god': 3,
 'creat': 4,
 'heaven': 5,
 'and': 6,
 'earth': 7,
 '.': 8,
 'wa': 9,
 'without': 10,
 'form': 11,
 ',': 12,
 'void': 13,
 ';': 14,
 'dark': 15,
 'upon': 16,
 'face': 17,
 'of': 18,
 'deep': 19,
 'spirit': 20,
 'move': 21,
 'water': 22,
 'said': 23,
 'let': 24,
 'there': 25,
 'be': 26,
 'light': 27,
 ':': 28,
 'saw': 29,
 'that': 30,
 'it': 31,
 'good': 32,
 'divid': 33,
 'from': 34,
 'call': 35,
 'day': 36,
 'he': 37,
 'night': 38,
 'even': 39,
 'morn': 40,
 'were': 41,
 'first': 42,
 'a': 43,
 'firmament': 44,
 'in': 45,
 'midst': 46,
 'made': 47,
 'which': 48,
 'under': 49,
 'abov': 50,
 'so': 51,
 'second': 52,
 'gather': 53,
 'togeth': 54,
 'unto': 55,
 'one': 56,
 'place': 57,
 'dri': 58,
 'land': 59,
 'appear': 60,
 'sea': 61,
 'bring': 62,
 'forth': 63,
 'grass': 64,
 'herb': 65,
 'yield': 66,
 'seed': 67,
 'fruit': 68,
 'tree': 69,
 'after': 70,
 'hi': 71,
 'kind': 72,
 'whose': 73,
 'is': 74,
 'itself': 75,
 'brought': 76,
 'third'

In [3]:
import random

rand = []
for r in range (6220):
            ra = random.randrange(0, 31100)
            rand.append(ra)
            
train_x = []
train_y = []
test_x = []
test_y = []
for i in range(len(data['text'])):
    if(i not in rand):
        train_x.append(i)
        train_y.append(i)
        
    elif(i in rand):
        test_x.append(i)
        test_y.append(i)

In [4]:
tf.compat.v1.disable_v2_behavior()

books = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges',
            'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles',
            'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
            'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel',
            'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk',
            'Zephaniah', 'Haggai',    'Zechariah',    'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians',
            '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians',
            '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
            'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
            'Revelation']

def encode(line):
    res_vec = np.zeros(66)
    idx = books.index(data.iloc[line]['book'])
    res_vec[idx] = 1
    return res_vec

def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

features = vocabulary_size
categories = 66 #number of books
epsilon = 1e-12
x = tf.compat.v1.placeholder(tf.float32,[None,features])
y_ = tf.compat.v1.placeholder(tf.float32,[None,categories])
W = tf.compat.v1.Variable(tf.zeros([features,categories]))
b = tf.compat.v1.Variable(tf.zeros([categories]))

y = tf.nn.softmax(tf.matmul(x,W)+b)
loss = -tf.reduce_mean(y_*tf.compat.v1.log(y))

update = tf.compat.v1.train.AdamOptimizer(0.0001).minimize(loss) #0.001

data_x = np.array([convert2vec(data.iloc[i]['text']) for i in train_x])
data_y = np.array([encode(i) for i in train_y])
test_data_x = np.array([convert2vec(data.iloc[i]['text']) for i in test_x])
test_data_y = np.array([encode(i) for i in test_y])
# data_x = np.array([convert2vec(data.iloc[i]['text']) for i in range(len(data['text']))])
# data_y = np.array([encode(i) for i in range(len(data['text']))])

sess = tf.compat.v1.Session()

sess.run(tf.compat.v1.global_variables_initializer())

for i in range(10000):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    if i % 100 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:data_x, y_:data_y}))
    

test1 = "In the beginning God created the heaven and the earth. "
t1 = np.array([convert2vec(test1)])
print("prediction for 1,1: " ,sess.run(y,feed_dict={x:t1}))

Instructions for updating:
non-resource variables are not supported in the long term
Iteration: 0  W: [[ 9.0189271e-05  8.4434345e-05 -1.8592929e-05 ... -8.8591558e-05
  -8.8591558e-05 -8.8591558e-05]
 [ 9.9958474e-05  9.9969999e-05  9.9955258e-05 ... -9.9932062e-05
  -9.9930541e-05  9.9382247e-05]
 [ 7.1295661e-05 -5.6231034e-05 -7.6013341e-05 ... -7.6013341e-05
  -7.6013341e-05  8.6211308e-05]
 ...
 [-2.7756967e-06 -2.7756967e-06 -2.7756967e-06 ... -2.7756967e-06
  -2.7756967e-06  6.4982458e-05]
 [-2.7756967e-06 -2.7756967e-06 -2.7756967e-06 ... -2.7756967e-06
  -2.7756967e-06  6.4982458e-05]
 [-2.7756967e-06 -2.7756967e-06 -2.7756967e-06 ... -2.7756967e-06
  -2.7756967e-06  6.4982458e-05]]  b: [ 9.9940742e-05  9.9914192e-05  9.9823388e-05  9.9921315e-05
  9.9868470e-05  9.9675250e-05  9.9514786e-05 -9.9831399e-05
  9.9811434e-05  9.9693971e-05  9.9787241e-05  9.9747784e-05
  9.9868121e-05  9.9815355e-05 -9.9665871e-05 -9.8789300e-05
 -9.9785851e-05  9.9895624e-05  9.9967052e-05  9.9

Iteration: 600  W: [[ 0.01307969 -0.0243496  -0.07680035 ... -0.04589102 -0.04613261
  -0.05832276]
 [ 0.02964172  0.04465891  0.04893224 ... -0.04403546 -0.04369348
   0.04901375]
 [-0.03055881 -0.07166396 -0.06411128 ... -0.04400625 -0.04414131
   0.05828508]
 ...
 [-0.02913411 -0.03348377 -0.03298107 ... -0.00872365 -0.0089599
   0.05609695]
 [-0.02913411 -0.03348377 -0.03298107 ... -0.00872365 -0.0089599
   0.05609695]
 [-0.02631078 -0.02824587 -0.02006692 ... -0.01087412 -0.01089491
   0.05698791]]  b: [ 0.04698044  0.04374785  0.04122817  0.04776808  0.03511277  0.03342152
 -0.02794723 -0.04584292  0.0167486   0.00919165  0.01998736  0.00994279
  0.05292059  0.02806753 -0.04714981 -0.05645313 -0.04698636  0.05644169
  0.0542178   0.05811213 -0.04298719 -0.04444074  0.0396798   0.03712
 -0.04351374  0.03811447 -0.0598242  -0.04488765 -0.04596943 -0.04517015
 -0.04693641 -0.04618276 -0.04586316 -0.04636892 -0.04600888 -0.04631495
 -0.04669032 -0.04684846 -0.04669107  0.04833956  0.

Iteration: 1300  W: [[-0.04429468 -0.10830916 -0.15275049 ... -0.08289909 -0.08446839
  -0.12218832]
 [ 0.01773802  0.07433347  0.08876086 ... -0.07559364 -0.07420507
   0.08629555]
 [-0.10750519 -0.14458425 -0.13536598 ... -0.07827287 -0.07936408
   0.12346762]
 ...
 [-0.0683869  -0.0865169  -0.08595229 ... -0.01254381 -0.01338404
   0.11252953]
 [-0.0683869  -0.0865169  -0.08595229 ... -0.01254381 -0.01338404
   0.11252953]
 [-0.06893513 -0.07145815 -0.04521618 ... -0.01985956 -0.01999019
   0.12064248]]  b: [ 0.08601712  0.07502374  0.0601419   0.08989016  0.04517186  0.0242286
 -0.11243921 -0.08374885 -0.0282794  -0.05299004 -0.02319394 -0.06149348
  0.10317247 -0.00281331 -0.0995368  -0.11863644 -0.09632534  0.1184854
  0.11206818  0.12194426 -0.08212875 -0.07772435  0.05093635  0.03946033
 -0.07667946  0.04313147 -0.12699951 -0.08597881 -0.08301535 -0.08689748
 -0.0853752  -0.08300318 -0.08583011 -0.08349826 -0.08238085 -0.08403786
 -0.08472758 -0.09744528 -0.08623338  0.09528529

Iteration: 2000  W: [[-0.09800276 -0.16992445 -0.20910941 ... -0.11352293 -0.11766323
  -0.18041211]
 [ 0.00400849  0.09217972  0.11863787 ... -0.09945114 -0.09635232
   0.09566081]
 [-0.15972804 -0.20592038 -0.20014858 ... -0.10613276 -0.10939614
   0.18490006]
 ...
 [-0.09641038 -0.12796448 -0.12362349 ... -0.0136141  -0.01484461
   0.16055521]
 [-0.09641038 -0.12796448 -0.12362349 ... -0.0136141  -0.01484461
   0.16055521]
 [-0.10476379 -0.10512178 -0.06109741 ... -0.02474422 -0.02504464
   0.17425337]]  b: [ 0.12160283  0.09914575  0.06645595  0.12745352  0.0438475  -0.01202306
 -0.18656148 -0.11982644 -0.09245635 -0.12410673 -0.08649477 -0.13831611
  0.14399074 -0.06160397 -0.14989495 -0.1738517  -0.14580078  0.17774154
  0.16832815  0.18093178 -0.12253491 -0.10694913  0.04545054  0.02753096
 -0.10741525  0.0332871  -0.18896131 -0.1291857  -0.11606056 -0.13126774
 -0.11779157 -0.11430039 -0.12574781 -0.11469706 -0.11317292 -0.117164
 -0.11727319 -0.15131854 -0.12272441  0.13937905

Iteration: 2700  W: [[-0.14749496 -0.21282391 -0.25037947 ... -0.1409507  -0.14891323
  -0.23420559]
 [-0.00345732  0.10289213  0.14341487 ... -0.11901923 -0.11347793
   0.09273256]
 [-0.19801347 -0.25968367 -0.26097125 ... -0.1308462  -0.1377466
   0.24246822]
 ...
 [-0.11941942 -0.161426   -0.1515567  ... -0.01400488 -0.01548603
   0.20911896]
 [-0.11941942 -0.161426   -0.1515567  ... -0.01400488 -0.01548603
   0.20911896]
 [-0.12965786 -0.12789512 -0.06972004 ... -0.02703167 -0.02750339
   0.21574849]]  b: [ 0.15498655  0.11870442  0.06954423  0.16386406  0.03885065 -0.05130174
 -0.2519853  -0.15828538 -0.15705085 -0.18948597 -0.14863119 -0.2081373
  0.17747709 -0.12339666 -0.19666097 -0.22077827 -0.1917781   0.23487946
  0.22315204  0.23500764 -0.16244496 -0.1368588   0.02903954  0.00784024
 -0.13795419  0.01641303 -0.24610817 -0.17474362 -0.15033796 -0.17775355
 -0.14765179 -0.14469253 -0.1687116  -0.14429124 -0.14307532 -0.1503543
 -0.14851826 -0.20463143 -0.16029596  0.18307275 

...

Iteration: 6800  W: [[-0.34093863 -0.18844967 -0.34121305 ... -0.29116568 -0.33725443
  -0.5175089 ]
 [ 0.01408465  0.12656581  0.23594265 ... -0.19702199 -0.17590101
   0.04932124]
 [-0.40170875 -0.47474802 -0.59799385 ... -0.26063114 -0.3176489
   0.5256832 ]
 ...
 [-0.19808385 -0.28038356 -0.24266103 ... -0.01432714 -0.01630806
   0.49153915]
 [-0.19808385 -0.28038356 -0.24266103 ... -0.01432714 -0.01630806
   0.49153915]
 [-0.17317131 -0.169434   -0.07945894 ... -0.02881439 -0.0296693
   0.35875303]]  b: [ 0.3204341   0.14516594  0.08074864  0.38850576  0.04195141 -0.2530549
 -0.57080334 -0.4040865  -0.50504154 -0.5025496  -0.46332005 -0.5593613
  0.22484644 -0.44636485 -0.4485584  -0.45260006 -0.43108475  0.55320317
  0.53354144  0.5090067  -0.4000027  -0.33205736 -0.20767131 -0.2212442
 -0.26476422 -0.13973792 -0.5508088  -0.46271437 -0.3885525  -0.45359048
 -0.33977792 -0.36253318 -0.4510771  -0.35319734 -0.36968747 -0.39015457
 -0.37230158 -0.50116926 -0.40568852  0.44500375  0

Iteration: 7500  W: [[-0.34537354 -0.18113172 -0.34711668 ... -0.31787083 -0.37144086
  -0.5650899 ]
 [ 0.0177718   0.1298994   0.24555877 ... -0.20796432 -0.18703283
   0.04139858]
 [-0.44074175 -0.49558073 -0.6556122  ... -0.28123823 -0.35049677
   0.56984663]
 ...
 [-0.20366582 -0.29437548 -0.25146356 ... -0.01432986 -0.0163329
   0.5378909 ]
 [-0.20366582 -0.29437548 -0.25146356 ... -0.01432986 -0.0163329
   0.5378909 ]
 [-0.17478411 -0.1709732  -0.07957139 ... -0.02882181 -0.0296847
   0.37968737]]  b: [ 0.34315294  0.13356057  0.07223905  0.4268316   0.04913699 -0.300992
 -0.62279004 -0.44722345 -0.56197345 -0.5527506  -0.5155322  -0.6169372
  0.19559813 -0.5000371  -0.49404094 -0.5040121  -0.47202063  0.6082047
  0.58695334  0.55482846 -0.44564882 -0.36966148 -0.26133093 -0.27243882
 -0.2809947  -0.17341228 -0.60303366 -0.51454234 -0.4315229  -0.50312454
 -0.37649012 -0.4055122  -0.50212127 -0.39337817 -0.41433927 -0.43439218
 -0.41371843 -0.5524755  -0.45046678  0.49173197  0.2

Iteration: 8200  W: [[-0.34002998 -0.18333192 -0.3542694  ... -0.34428364 -0.40552953
  -0.61273783]
 [ 0.02130452  0.13329236  0.25424793 ... -0.21806073 -0.19996537
   0.03376313]
 [-0.47910354 -0.5091566  -0.7135678  ... -0.30037266 -0.3825366
   0.6136162 ]
 ...
 [-0.20777701 -0.30701885 -0.2591568  ... -0.01433126 -0.01635048
   0.5844659 ]
 [-0.20777701 -0.30701885 -0.2591568  ... -0.01433126 -0.01635048
   0.5844659 ]
 [-0.17592542 -0.17200732 -0.07962772 ... -0.02882493 -0.02969231
   0.40101022]]  b: [ 0.3635524   0.11860903  0.05825998  0.46404985  0.05624718 -0.354511
 -0.6751286  -0.490861   -0.61876607 -0.6037077  -0.5683015  -0.6746257
  0.154109   -0.55432296 -0.5410013  -0.55995363 -0.513685    0.66378665
  0.64073867  0.6010772  -0.49337345 -0.40903166 -0.31634235 -0.32472447
 -0.29781112 -0.20920257 -0.6559904  -0.56737643 -0.4754282  -0.55368227
 -0.4137828  -0.44977912 -0.5537484  -0.4344552  -0.45992842 -0.4791647
 -0.4556214  -0.604294   -0.496113    0.53942835  0

Iteration: 8900  W: [[-0.32656437 -0.19319771 -0.36410564 ... -0.36998656 -0.43904275
  -0.660374  ]
 [ 0.02501956  0.13661575  0.26244313 ... -0.22708769 -0.21493784
   0.02650445]
 [-0.51736444 -0.5134898  -0.7718008  ... -0.31774324 -0.4132228
   0.6571214 ]
 ...
 [-0.2107757  -0.31842068 -0.26597342 ... -0.01433191 -0.01636341
   0.6312977 ]
 [-0.2107757  -0.31842068 -0.26597342 ... -0.01433191 -0.01636341
   0.6312977 ]
 [-0.17673716 -0.17270114 -0.07965542 ... -0.02882624 -0.02969616
   0.42266753]]  b: [ 0.38101283  0.10119274  0.03875227  0.499684    0.06248834 -0.41225442
 -0.72794735 -0.53513086 -0.6755963  -0.6556393  -0.6217401  -0.7325127
  0.10359911 -0.6093872  -0.5894967  -0.61912525 -0.55616844  0.7198696
  0.69490564  0.6478356  -0.5429973  -0.45020577 -0.37211856 -0.37763608
 -0.31575948 -0.24688616 -0.7096722  -0.62123847 -0.5202844  -0.6052331
 -0.45168865 -0.4951788  -0.6059055  -0.476378   -0.50640726 -0.5245436
 -0.49809653 -0.656668   -0.5424518   0.5879854   0

Iteration: 9600  W: [[-0.30689514 -0.20768924 -0.37731847 ... -0.3946577  -0.47157812
  -0.707912  ]
 [ 0.02903694  0.13974115  0.27050936 ... -0.23510647 -0.232105
   0.01973087]
 [-0.5556411  -0.5071328  -0.8302571  ... -0.33325055 -0.4421774
   0.70039356]
 ...
 [-0.21296217 -0.3287417  -0.27212563 ... -0.01433232 -0.01637338
   0.67834383]
 [-0.21296217 -0.3287417  -0.27212563 ... -0.01433232 -0.01637338
   0.67834383]
 [-0.17731915 -0.17316845 -0.07966879 ... -0.02882656 -0.02969815
   0.44446063]]  b: [ 0.3948091   0.08200166  0.01410329  0.53322     0.06758726 -0.47288755
 -0.7813084  -0.58010846 -0.73261136 -0.70857716 -0.675884   -0.79065526
  0.04765223 -0.66529363 -0.6394196  -0.680232   -0.59948754  0.7763495
  0.74945706  0.6952053  -0.59421086 -0.49304143 -0.4281968  -0.43092817
 -0.33498418 -0.28601176 -0.76399463 -0.67604846 -0.56600034 -0.6577208
 -0.4902451  -0.54154575 -0.65856934 -0.5190946  -0.5537153  -0.57059723
 -0.5412165  -0.7096205  -0.58932155  0.63721097  0

we now encoded every book to one hot vector.
also we have the bag of words.

right now we see that our model is weak almost all numbers are equal, so we need to improve our model,
but after alot of epochs, the correct prediction get a little more points than other. 

In [5]:
data_x

array([[0., 6., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       ...,
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
data_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])